In [51]:
import os
import json
from dotenv import load_dotenv

load_dotenv()

from langchain_teddynote import logging
logging.langsmith("aid-backend")

from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# notices.json 파일을 읽고 input_notice에 할당
def load_notices():
    with open("../notices.json", "r", encoding="utf-8") as file:
        data = json.load(file)
    return data

# notices.json 내용을 input_notice에 넣기
input_notice = load_notices()

# Prompt 설정
template = """
장학금 DB를 구축해야 합니다.
각 필드는 아래 주의사항에 따라 주어진 형식에 맞게 반환해 주세요. 
조건이 맞지 않거나 해당 정보가 없으면 null로 처리해 주세요. 모든 항목이 누락 없이 반환되어야 합니다.

주의사항 및 파싱 방법:
1. `scholarshipID`: 반드시 고유한 정수 값이어야 합니다.
2. `eligibleMajors`: 대상 학과 목록이 없을 경우 null로 설정합니다.
3. `minimumGPARequirement`: 최소 학점이 정해져 있지 않은 경우 null로 설정합니다.
4. `eligibleSemesters`: 반드시 학년이랑 학기를 구별해야 합니다. 예를 들어 2~3학년 재학생인 경우 [4,5,6,7]가 입력되어야 합니다. (1학년=1,2학기/2학년=3,4학기/3학년=5,6학기/4학년=7,8학기) 대학원 재학생일 경우 [-1]로 표시합니다. 학부생 대상인데 별다른 언급이 없는 경우 [1, 2, 3, 4, 5, 6, 7, 8] 이 들어갑니다.
5. `scholarshipType`: 성적 우수 장학금, 생활비 지원 장학금, 등록금성 장학금 등의 유형을 명시해 주세요.
6. `ageLimit`: 나이 제한이 없으면 null로 설정합니다.
7. `regionalRestrictions`: 특정 지역에만 지원 가능하다면 해당 지역을 배열로 넣고, 제한이 없다면 null로 설정합니다.
8. `incomeLevelRequirement`: 소득 분위 제한이 없는 경우 null로 설정합니다. 가정 형편이 어려운 학생이 기준일 경우 3으로 표시합니다.
9. `applicationPeriod`: "시작일 ~ 종료일" 형식으로 작성합니다. 시작일은 최종 수정일 기준입니다.
10. `scholarshipAmount`: 장학금 금액이 정해져 있지 않으면 null로 설정합니다. 구체적으로 장학금 지금액이 얼마나 되는지 string 형태로 작성해주세요.
11. `numberOfRecipients`: 선발 인원이 정해져 있지 않으면 null로 설정합니다.
12. `requiredDocuments`: 제출 서류가 없으면 null로 설정합니다.
13. `applicationMethod`: 온라인 신청, 우편 접수 등 구체적인 방법을 기재합니다.
14. `significant`: 행사 참석 필수 등 특이사항이 없으면 null로 설정합니다.
15. `link`: 장학금 안내 링크를 올바른 URL 형식으로 작성합니다.
16. `views`: 숫자 형식의 조회수를 포함해 주세요.
17. `foundation`: 재단 이름이 없으면 null로 설정합니다.

정보:
{input_notice}

FORMAT:
{{
    "scholarshipID": "integer",                       // 장학금 고유 ID 번호
    "scholarshipName": "string",                      // 장학금 이름
    "eligibleMajors": "array or null",                // 지원 대상 학과 배열, 정보가 없으면 null
    "minimumGPARequirement": "number or null",        // 최소 학점 기준, 정보가 없으면 null
    "eligibleSemesters": "array",                     // 지원 가능 학기 배열, 학부의 경우 숫자 배열로, 대학원 재학생일 경우 [-1]
    "scholarshipType": "string",                      // 장학금 유형
    "ageLimit": "integer or null",                    // 나이 제한, 정보가 없으면 null
    "regionalRestrictions": "array or null",          // 지역 제한, 정보가 없으면 null
    "incomeLevelRequirement": "number or null",       // 소득분위 기준, 정보가 없으면 null
    "applicationPeriod": "string",                    // 신청 기간 (예: "2024-01-01 ~ 2024-12-31")
    "scholarshipAmount": "string or null",            // 장학금 금액, 정보가 없으면 null
    "numberOfRecipients": "integer or null",          // 선발 인원, 정보가 없으면 null
    "requiredDocuments": "array or null",             // 제출 서류 배열, 정보가 없으면 null
    "applicationMethod": "string",                    // 신청 방법
    "significant": "string or null",                  // 특이사항, 정보가 없으면 null
    "link": "string",                                 // 장학금 안내 링크
    "views": "integer",                               // 조회수
    "foundation": "string or null"                    // 재단 이름, 정보가 없으면 null
}}
"""

# 나머지 코드는 동일
prompt_template = PromptTemplate(input_variables=["input_notice"], template=template)

# 모델과 체인 설정
model = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4o-mini",
)

output_parser = StrOutputParser()

# 체인 생성
formatted_prompt = prompt_template.format(input_notice=input_notice)
response = model.invoke(formatted_prompt)


prompt_template = PromptTemplate(input_variables=["input_notice"], template=template)

# 모델과 체인 설정
model = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4o-mini",
)

output_parser = StrOutputParser()

# 체인 생성
formatted_prompt = prompt_template.format(input_notice=input_notice)
response = model.invoke(formatted_prompt)

LangSmith 추적을 시작합니다.
[프로젝트명]
aid-backend


In [55]:
import re

response_text = response.content.strip()
if response_text.startswith("```json"):
    response_text = response_text[7:]  # ```json 제거
if response_text.endswith("```"):
    response_text = response_text[:-3]  # ``` 제거

try:
    answer_json = json.loads(response_text)
    with open("response.json", "w", encoding="utf-8") as file:
        json.dump(answer_json, file, ensure_ascii=False, indent=4)
    print("JSON 파일로 저장되었습니다.")
except json.JSONDecodeError as e:
    print("JSONDecodeError 발생:", e)
    print("응답을 올바른 JSON 형식으로 전처리 후 다시 시도하세요.")


JSON 파일로 저장되었습니다.


In [58]:
import os
import json
from pymongo import MongoClient
from pymongo.server_api import ServerApi

uri = os.getenv("MONGO_URI")
client = MongoClient(uri, server_api=ServerApi('1'))

with open("./response.json", "r", encoding="utf-8") as file:
    scholarship_data = json.load(file)

try:
    client.admin.command('ping')
    print("MongoDB에 성공적으로 연결되었습니다!")

    db = client["db"]
    collection = db["scholarships"]

    for scholarship in scholarship_data:
        scholarship["_id"] = scholarship.pop("scholarshipID")

    if isinstance(scholarship_data, list):
        collection.insert_many(scholarship_data)
        print("데이터가 성공적으로 저장되었습니다!")
    else:
        collection.insert_one(scholarship_data)
        print("데이터가 성공적으로 저장되었습니다!")
finally:
    # 클라이언트 연결 종료
    client.close()


MongoDB에 성공적으로 연결되었습니다!
데이터가 성공적으로 저장되었습니다!
